In [27]:
import re
import json
import numpy as np
import os
def extract_required_entities(path):
    required_entities=[]
    intents=os.listdir(path) 
    intents= list(filter(lambda name: len(re.findall("_[A-Za-z][A-Za-z].json$",name))==0,intents))
    for intent in intents:
        with open(path+intent) as f:
            data = json.load(f)
        parameters=data["responses"][0]["parameters"]
        for param in parameters:
            if param["required"]== True:
                print(param["name"],intent)
                required_entities.append(param["name"])
                
            
    
            
        
    
    return np.unique(required_entities)

path_intents='dialogflow/intents/'
entities=extract_required_entities(path_intents)

doremus-artist discover-artist.json
date-period find-performance.json
doremus-artist works-by-artist.json
doremus-genre works-by-genre.json
doremus-instrument works-by-instrument.json
date-period works-by-years.json


In [28]:
entities

array(['date-period', 'doremus-artist', 'doremus-genre',
       'doremus-instrument'], dtype='<U18')

In [73]:
required_entities=['date-period', 'doremus-artist', 'doremus-genre',
       'doremus-instrument']
entities_to_intents={'date-period': 'inform_period','doremus-artist': 'inform_artist', 
                     'doremus-genre': 'inform_genre', 'doremus-instrument': 'inform_instrument'}

def create_new_intents(path):
    new_intent_examples_list=[]
    with open(path,encoding="utf8") as f:
        data = json.load(f)
    common_examples=data["rasa_nlu_data"]["common_examples"]
    for example in common_examples[:100]:
        if 'entities' in example:
            for entity in example['entities']:
                if entity['entity'] in required_entities:
                    new_intent_example={}
                    length_entity=entity['end']-entity['start']
                    new_intent_example['intent']=entities_to_intents[entity['entity']]
                    new_intent_example['entities']=[{'start': 0, 'end': length_entity, 'value': entity['value'], 
                                                      'entity': entity['entity']}]
                    new_intent_example['text'] = entity['value']
                    new_intent_examples_list.append(new_intent_example)
    new_intent_examples_json={}
    new_intent_examples_json[ "rasa_nlu_data"]={"common_examples": new_intent_examples_list }
    with open('data/new_intents.json', 'w') as f:
        json.dump(new_intent_examples_json,f, indent=2)

            
            
create_new_intents("data/training_data.json")

In [ ]:
def create_int